In [ ]:
## Run this only if running in colab
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


## Imports

In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 686kB 3.7MB/s 


In [ ]:
import cv2
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab.patches import cv2_imshow
import PIL
import tensorflow_addons as tfa

In [ ]:
def add_salt_pepper_noise(X_imgs):
    X_img = X_imgs.numpy().copy()
    row, col, _ = X_img.shape
    salt_vs_pepper = 0.2
    amount = 0.004
    num_salt = np.ceil(amount * X_img.size * salt_vs_pepper)
    num_pepper = np.ceil(amount * X_img.size * (1.0 - salt_vs_pepper))
    
    # Add Salt noise
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in X_img.shape]
    X_img[coords[0], coords[1], :] = 1

    # Add Pepper noise
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in X_img.shape]
    X_img[coords[0], coords[1], :] = 0
    return tf.convert_to_tensor(X_img, dtype=tf.uint8)

## Defining Augmenting Function


In [ ]:
def augment_images(dir, aug_dir): 
    IMG_SIZE = 256
    for img_path in os.listdir(dir):
        image = tf.keras.preprocessing.image.load_img(dir+img_path)
        img_tensor = tf.convert_to_tensor(tf.keras.preprocessing.image.img_to_array(image))
        
        img_tensor = tf.cast(img_tensor, tf.uint8)

        img_tensor = tf.image.resize(img_tensor, (IMG_SIZE,IMG_SIZE))

        fliplr = tf.image.flip_left_right(img_tensor)
        flipud = tf.image.flip_up_down(img_tensor)
        rotl = tfa.image.rotate(img_tensor, angles=3.1415/9)
        rotr = tfa.image.rotate(img_tensor, angles=-3.1415/9)
        
        noise = tf.random.normal(shape=tf.shape(img_tensor), mean=0.0, stddev=1.0)
        noise_img = tf.add(tf.cast(img_tensor, tf.uint8), tf.cast(noise,dtype=tf.uint8))

        sp_img = add_salt_pepper_noise(img_tensor)
        
        tf.keras.preprocessing.image.save_img(aug_dir+img_path, img_tensor)    # save original
        tf.keras.preprocessing.image.save_img(aug_dir+img_path.split(".")[0]+"_fliplr.png", fliplr)
        tf.keras.preprocessing.image.save_img(aug_dir+img_path.split(".")[0]+"_flipud.png", flipud)
        tf.keras.preprocessing.image.save_img(aug_dir+img_path.split(".")[0]+"_rotl.png", rotl)
        tf.keras.preprocessing.image.save_img(aug_dir+img_path.split(".")[0]+"_rotr.png", rotr)
        tf.keras.preprocessing.image.save_img(aug_dir+img_path.split(".")[0]+"_noise.png", noise_img)
        tf.keras.preprocessing.image.save_img(aug_dir+img_path.split(".")[0]+"_sp.png", sp_img)

        print("Augmenting image {}".format(dir+img_path))

In [ ]:
data_mining_dir = '/gdrive/MyDrive/Data Mining - 255/'
covid_dataset_dir = data_mining_dir + '/Dataset/COVID/'
noncovid_dataset_dir = data_mining_dir + '/Dataset/non-COVID/'

aug_covid_dataset_dir = data_mining_dir + 'Augmented Dataset/COVID/'
aug_noncovid_dataset_dir = data_mining_dir + 'Augmented Dataset/non-COVID/'

# Augmenting NON COVID images
print("Total NON COVID Images before Augmentation = {}".format(len(os.listdir(noncovid_dataset_dir))))
# augment_images(noncovid_dataset_dir, aug_noncovid_dataset_dir)
print("Total NON COVID Images after Augmentation = {}".format(len(os.listdir(aug_noncovid_dataset_dir))))

# Augmenting COVID images
print("Total COVID Images before Augmentation = {}".format(len(os.listdir(covid_dataset_dir))))
# augment_images(covid_dataset_dir, aug_covid_dataset_dir)
print("Total COVID Images after Augmentation = {}".format(len(os.listdir(aug_covid_dataset_dir))))

Total NON COVID Images before Augmentation = 1019
Total NON COVID Images after Augmentation = 10428
Total COVID Images before Augmentation = 1381
Total COVID Images after Augmentation = 9198
